In [13]:
import pickle
import csv
import os
import datetime as dt
import pandas as pd
import pandas_datareader.data as pdr
import numpy as np
import bs4 as bs
import requests

# Due to yahoo finance discontinueing their API, had to use a work around
# this package uses deprecated functions
# import fix_yahoo_finance as yf
# yf.pdr_override()

In [35]:
def save_sp500_tickers(fn = "wiki_sp500tickers.pickle", save_pickle=True, verbose=False):
    if verbose:
        print("in save tickers")
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    wiki_table = soup.find('table', {'class': 'wikitable sortable'})
    
    # now we have the wiki table with all the data we want the tickers to pull from yahoo/google
    wiki_tickers = []
    for row in wiki_table.findAll('tr')[1:]:     # dont need header column
        ticker = row.findAll('td')[0].text      # grab table data text
        wiki_tickers.append(ticker)
    
    # save list
    if verbose:
        print("Tickers have been found saving to disk")
    if save_pickle:
        with open(fn, "wb") as f:
            pickle.dump(wiki_tickers, f)
    else:
        with open(fn, "w") as f:
            wr = csv.writer(f)
            wr.writerow(["Ticker#", "Ticker"])
            count = 0
            for ticker in wiki_tickers:
                wr.writerow([count, ticker])
                count += 1
    
    return wiki_tickers

In [42]:
def get_data_from_online(repull_sp500=False, ticker_fn='wiki_sp500tickers.pickle', save_pickle=True, finance_source='google', verbose=False):
    if verbose:
        print('Loading tickers')
        
    if repull_sp500:
        tickers = save_sp500_tickers(ticker_fn, save_pickle, verbose)
    # read sp500 list from disk
    else:
        with open(ticker_fn, "rb") as f:
            if save_pickle:
                tickers = pickle.load(f)
            else:
                tickers = []
                rd = csv.reader(f, deliminter='')
                for row in rd:
                    tickers.append(row)
         
    # make directory if not made        
    if not os.path.exists("data/stock_data"):
        if verbose:
            print("making stock_data dir")
        os.makedirs("data/stock_data")
    
    # data dates 
    start = dt.datetime(2000, 1, 1)
    end = dt.datetime.today()
    stocks_not_found = []
    stocks_not_found_fn = "missing_stocks_{}".format(finance_source)
    
    if verbose:
        print("saving stock data to file")
        
    for ticker in tickers:
        if not os.path.exists('data/stock_data/{}'.format(ticker)):
            try:
                df = pdr.DataReader(ticker, finance_source, start, end)
                df.to_csv('data/stock_data/{}.csv'.format(ticker))
            
            except:
                print("had a connection error and skipped {}".format(ticker))
                stocks_not_found.append(ticker)
        else:
            print('did not replace {}'.format(ticker))
            
    if verbose:
        print("finished with saving stock data")

    with open(stocks_not_found_fn, "wb") as f:
        pickle.dump(stocks_not_found, f)


In [45]:
# get_data_from_online(verbose=True)

In [46]:
def combine_sp500_data(tickers_fn="wiki_sp500tickers.pickle", combined_fn="allSP500_closed_google.csv", 
                       missing_stocks_fn='missing_stocks_google.pickle'):
    
    with open(tickers_fn, "rb") as f:
        tickers = pickle.load(f)

    with open(missing_stocks_fn, "rb") as f:
        missing_tickers = pickle.load(f)
        
    sp500_df = pd.DataFrame()
    for count, ticker in enumerate(tickers):
        if count % 10 ==0:
            print(count)
            
        if ticker in missing_tickers:
            print("ticker {} has been skipped".format(ticker))
            continue
            
        try:
            df = pd.read_csv('data/stock_data/{}.csv'.format(ticker))
            df.set_index('Date', inplace=True)
            
            # Change close column name to ticker and drop other columns
            df.rename(columns={'Close':ticker}, inplace=True)
            df.drop(["Open", "High", "Low", "Volume"], 1, inplace=True)
            
            if sp500_df.empty:
               sp500_df = df
            else:
                sp500_df = sp500_df.join(df, how="outer")
        except:
            raise 
            
    sp500_df.tocsv('data/{}'.format(combined_fn))

In [47]:
combine_sp500_data()

FileNotFoundError: [Errno 2] No such file or directory: 'missing_stocks_google.pickle'

In [40]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2013, 1, 27)
f = web.DataReader("LMT", 'google', start, end)
f.head()

RemoteDataError: Unable to read URL: http://www.google.com/finance/historical?startdate=Jan+01%2C+2010&output=csv&enddate=Jan+27%2C+2013&q=LMT